In [1]:
import numpy as np
import os

import plotly.graph_objs as go

from trProcess import *

In [3]:
tracenum = 10
baselinelist = {
    "FqCoDel": ["FqCoDel_0_5ms1000", "FqCoDel_0_500ms1000"], 
    "FIFO": ["PfifoFast_0_1000"],
    # "CoDel": ["CoDel_0_5ms1000"],
    "Choke": ["Choke_0_1000"],
    "Hhf": ["Hhf_0_1000"],
    "Sfb": ["Sfb_0_1000"],
    # "DWRR (Class, mislabeled)": ["Dwrr_19_1.00", "Dwrr_29_1.00", "Dwrr_39_1.00"],
    "Auto (Class, online)": ["Auto_0_1.00_ExpClass5.00"],
    "DualQ": ["DualQ_9_1000"],
    "DWRR (Class, prelabeled)": ["Dwrr_9_1.00"],
    "StrictiPriotity": ["StrictPriority_90_1000", "StrictPriority_9_1000"],
}

indir = "ns-3.34/logs/"
flowsizes = [10000]

for exp in ["46_Video1_Ftp10000003"]:
    apps = exp.split("_")
    data1, data2 = [], []
    for name, baselines in baselinelist.items():
        p90Fct, maxAppA, maxAppB, jfi = [], [], [], []
        for baseline in baselines:
            p90FctTrace, maxAppATrace, maxAppBTrace, jfiTrace = [], [], [], []
            for traceid in range(tracenum):
                trace = "maxwell_30.0_0.01_300_%d.pitree-trace" % traceid
                conf = os.path.join (indir, exp, baseline, trace)
                    
                curMaxApp, curAppDur, curAvgApp = appTraceProcessor (os.path.join (conf, 'app.tr'))
     
                curAvgFct, curP90Fct, fctLastStart, fctFirstEnd, fcts = fctTraceProcessor(os.path.join (conf, 'fct.tr'))
                bws = 8 * flowsizes[0] / np.array (fcts)
                isRto, _, _, avgBw = bwTraceProcessor(os.path.join (conf, 'bw.tr'), fctLastStart, fctFirstEnd)
                curAvgAppFlows = [avgApp for port, avgApp in curAvgApp.items() if port in range (10000, 20000) and isRto[port] < 100]
                if len (curAvgAppFlows) > 0:
                    maxAppATrace.append (np.mean (curAvgAppFlows))
                    for port, bw in avgBw.items():
                        if port in range(10000, 30000):
                            bws = np.append (bws, bw / 1e6)
                    jfiTrace.append (pow(np.nansum(bws),2) / (bws.size * np.nansum(np.square(bws))))
            [maxAppATrace, jfiTrace] = RemoveOutliers ([maxAppATrace, jfiTrace])
            maxAppA.append(np.mean(maxAppATrace))
            jfi.append(np.mean(jfiTrace))
            print(maxAppA[-1], jfi[-1], baseline)
                
        data1.append(go.Scatter(
            x=maxAppA,
            y=jfi,
            name=name
        ))
    fig = go.Figure(data=data1)
    fig.update_layout(
        title=exp,
        xaxis_title="Average Video Latency (ms)",
        yaxis_title="JFI"
    )
    fig.show()

64.71111111111111 0.9957547560602347 FqCoDel_0_5ms1000
62.0 0.9933753387353621 FqCoDel_0_500ms1000
1027.7599999999995 0.7975379554720096 PfifoFast_0_1000
69.49999999999999 0.8005696572153285 Choke_0_1000
62.0 0.7743078907815615 Hhf_0_1000
434.45 0.7696316597943746 Sfb_0_1000
62.0 0.9919407386476445 Auto_0_1.00_ExpClass5.00
62.5 0.6858585747133302 DualQ_9_1000
62.0 0.9437920547247135 Dwrr_9_1.00


/home/mengzl/miniconda3/envs/diffqm/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning:

Mean of empty slice.

/home/mengzl/miniconda3/envs/diffqm/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning:

invalid value encountered in double_scalars

/home/mengzl/miniconda3/envs/diffqm/lib/python3.10/site-packages/numpy/core/_methods.py:262: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/mengzl/miniconda3/envs/diffqm/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning:

invalid value encountered in true_divide

/home/mengzl/miniconda3/envs/diffqm/lib/python3.10/site-packages/numpy/core/_methods.py:254: RuntimeWarning:

invalid value encountered in double_scalars



nan nan StrictPriority_90_1000
62.2 0.6385326192700236 StrictPriority_9_1000
